In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
import pandas as pd 
import pyperclip
import time



# 웹 드라이버 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com")
time.sleep(2)


user_id = "ffjjo0310"
user_pw = "wnsgud800320"

# 2. 아이디 입력
id = driver.find_element(By.CSS_SELECTOR, "#id")
pyperclip.copy(user_id)
id.send_keys(Keys.CONTROL, 'v')
time.sleep(2)

# 3. 비밀번호 입력
pw = driver.find_element(By.CSS_SELECTOR, "#pw")
pyperclip.copy(user_pw)
pw.send_keys(Keys.CONTROL, 'v')
time.sleep(2)

# 4. 로그인 버튼 클릭
driver.find_element(By.CSS_SELECTOR, ".btn_login").click()

# 네이버 중고나라 카페 접속
driver.get("https://cafe.naver.com/joonggonara")

# 특정 메뉴(예: '전체 글') 클릭
search_menu = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "#menuLink339"))
)


search_menu.click()

# iframe으로 전환
driver.switch_to.frame("cafe_main")

# 검색 박스 찾기
search_box = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "query"))
)

# 검색 박스에 '아이폰' 입력하고 엔터키 입력
search_box.send_keys("아이폰")
search_box.send_keys(Keys.ENTER)

# 목록 크기 선택
list_size_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "listSizeSelectDiv"))
)
list_size_button.click()

# "50개씩 보기" 선택
fifty_per_page_option = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "#listSizeSelectDiv > ul > li:nth-child(7) > a"))
)
fifty_per_page_option.click()


# 검색 옵션 드롭다운 메뉴 활성화
search_option_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "searchOptionSelectDiv"))
)
search_option_button.click()

# "안전거래" 선택을 위한 명시적 대기
WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.CSS_SELECTOR, "#searchOptionSelectDiv > ul"))
)

# "안전거래" 선택
safe_transaction_option = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "#searchOptionSelectDiv > ul > li:nth-child(3) > a"))
)
safe_transaction_option.click()



data = []

# 페이지 이동 로직 및 데이터 추출
for page in range(1, 81):  # 1부터 80페이지까지
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, ".article"))
    )
    num_posts = len(driver.find_elements(By.CSS_SELECTOR, ".article"))
    print(f"Processing page {page} with {num_posts} posts...")

    for i in range(num_posts):
        try:
            # 매번 새로운 요소를 찾아 클릭
            post_links = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".article"))
            )
            driver.execute_script("arguments[0].click();", post_links[i])

            # 데이터 추출
            model_name = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".ProductName"))
            ).text
            date = driver.find_element(By.CSS_SELECTOR, ".date").text
            price = driver.find_element(By.CSS_SELECTOR, ".ProductPrice").text




               # 예외 처리 추가
            try:
                info = driver.find_element(By.CSS_SELECTOR, ".se-module.se-module-text").text
            except NoSuchElementException:
                print("Info element not found, using default info.")
                info = "No additional information available."



            data.append([model_name, date, price, info])

            # 뒤로 가기
            driver.back()
            WebDriverWait(driver, 10).until(
                EC.frame_to_be_available_and_switch_to_it((By.ID, "cafe_main"))
            )
        except StaleElementReferenceException:
            print("Element has become stale, retrying...")
            continue
        except TimeoutException:
            print("Time out while trying to access the post details.")
            continue

    if page % 10 == 0:  # 10, 20, 30... 페이지에서 다음 버튼 클릭
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "#main-area > div.prev-next > a.pgR > span"))
            )
            next_button.click()
            print(f"Moved to next set of pages after page {page}")
        except TimeoutException:
            print("Time out while trying to click next button.")
            continue
        except NoSuchElementException: # type: ignore
            print("Next button not found. It might be the last page.")
            break

    # 다음 페이지로 이동
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.LINK_TEXT, str(page + 1)))
        ).click()
        print(f"Moved to page {page + 1}")
    except TimeoutException:
        print("Time out while trying to navigate to the next page.")
        break

# 데이터 저장
df = pd.DataFrame(data, columns=['Model Name', 'Date', 'Price', 'Information'])
df.to_csv('iphone_data.csv', index=False, encoding='utf-8-sig')

# 웹 드라이버 종료
driver.quit()

In [4]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv('iphone_data.csv')

# 데이터 중복 확인
print("원본 데이터의 행 수:", len(df))
print("중복 제거 후의 행 수:", df.drop_duplicates().shape[0])

# 중복된 행 제거
df_cleaned = df.drop_duplicates()

# 중복 제거된 데이터를 새로운 CSV 파일로 저장
df_cleaned.to_csv('iphone_data_cleaned.csv', index=False, encoding='utf-8-sig')

print("중복 제거된 데이터가 'iphone_data_cleaned.csv' 파일로 저장되었습니다.")


원본 데이터의 행 수: 4000
중복 제거 후의 행 수: 2043
중복 제거된 데이터가 'iphone_data_cleaned.csv' 파일로 저장되었습니다.


In [5]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv('iphone_data.csv')

# 'Model Name' 컬럼에서 '아이폰' 또는 'iphone'이 포함된 행만 유지
df_filtered = df[df['Model Name'].str.contains('아이폰|iphone', case=False, na=False)]

# 필터링된 데이터를 새로운 CSV 파일로 저장
df_filtered.to_csv('iphone_data_filtered.csv', index=False, encoding='utf-8-sig')

print("모델명에 '아이폰' 또는 'iphone'이 들어가지 않은 행을 제거하고 'iphone_data_filtered.csv' 파일로 저장되었습니다.")


모델명에 '아이폰' 또는 'iphone'이 들어가지 않은 행을 제거하고 'iphone_data_filtered.csv' 파일로 저장되었습니다.


In [1]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv('iphone_data.csv')

# 제거할 텍스트 정의
remove_text1 = "📢 제목에 \"제조사/ 브랜드 명\"과 “상품명(ex. 갤럭시 S11) ”을 넣어 작성하면, 보다 빠른 판매가 가능합니다!"
remove_text2 = "📢 게시글 작성 시 배송 방법에 “직거래”와 “내 위치” 설정할 경우, 보다 빠른 판매가 가능합니다!"
remove_text3 = "※ 등록한 게시글이 회원의 신고를 받거나 이상거래로 모니터링 될 경우 중고나라 사기통합조회 DB로 수집/활용될 수 있습니다."


# 'Information' 컬럼에서 지정한 텍스트 제거
df['Information'] = df['Information'].str.replace(remove_text1, '', regex=False)
df['Information'] = df['Information'].str.replace(remove_text2, '', regex=False)
df['Information'] = df['Information'].str.replace(remove_text3, '', regex=False)
# 정리된 데이터를 새로운 CSV 파일로 저장
df.to_csv('iphone_data_final.csv', index=False, encoding='utf-8-sig')

print("정보에서 불필요한 텍스트를 제거하고 'iphone_data_final.csv' 파일로 저장되었습니다.")


정보에서 불필요한 텍스트를 제거하고 'iphone_data_1.csv' 파일로 저장되었습니다.


In [ ]:
import pandas as pd
import re

def load_and_preprocess_data(file_path):
    # 데이터 로드
    data = pd.read_csv(file_path)

    # 'Date' 칼럼을 datetime 객체로 변환
    data['Date'] = pd.to_datetime(data['Date'], format='%Y.%m.%d. %H:%M')

    # 'Price' 칼럼에서 숫자만 추출하여 정수형으로 변환
    data['Price'] = data['Price'].str.replace('원', '').str.replace(',', '').astype(int)

    # 'Capacity' 정보 추출
    def extract_capacity(text):
        match = re.search(r'(\d+(GB|G|기가))\b', text)
        if match:
            return match.group(1)
        return 'Unknown'

    data['Capacity'] = data['Information'].astype(str).apply(extract_capacity)

    return data

def save_data(data, save_path):
    # 데이터를 UTF-8 인코딩으로 CSV 파일로 저장
    data.to_csv(save_path, index=False, encoding='utf-8')

# 사용 예
file_path = 'path_to_your_data.csv'  # 데이터 파일 경로
processed_save_path = 'path_to_save_processed_data.csv'  # 저장할 파일 경로

data = load_and_preprocess_data(file_path)
save_data(data, processed_save_path)


In [3]:
import pandas as pd
import re

def load_and_preprocess_data(file_path):
    # 데이터 로드
    data = pd.read_csv(file_path)

    # 'Date' 칼럼을 datetime 객체로 변환
    data['Date'] = pd.to_datetime(data['Date'], format='%Y.%m.%d. %H:%M')

    # 'Price' 칼럼에서 숫자만 추출하여 정수형으로 변환
    data['Price'] = data['Price'].str.replace('원', '').str.replace(',', '').astype(int)

    # 'Capacity' 정보 추출
    def extract_capacity(text):
        match = re.search(r'(\d+(GB|G|기가))\b', text)
        if match:
            return match.group(1)
        return 'Unknown'

    data['Capacity'] = data['Information'].astype(str).apply(extract_capacity)

    return data

def save_data(data, save_path):
    # 데이터를 UTF-8 인코딩으로 CSV 파일로 저장
    data.to_csv(save_path, index=False, encoding='utf-8')

# 사용 예
file_path = r'C:\startcoding\python_crawling\Capstone\iphone_data_final.csv'  # 데이터 파일 경로
processed_save_path = r'C:\startcoding\python_crawling\Capstone\processed_iphone_data.csv'  # 저장할 파일 경로

data = load_and_preprocess_data(file_path)
save_data(data, processed_save_path)


In [5]:
pip install -r requirements.txt


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
